# v2.0
### This version 2.0 uses dNBR map and otsu's thresholded map as input to SVM


In [21]:
import geemap
import ee

In [22]:
ee.Initialize()

In [23]:
landsatMap = geemap.Map()

In [24]:
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

## PREFIRE

In [25]:
point = ee.Geometry.Point([-122.8, 39.87])

landsat_prefire_image = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(point)
    .filterDate('2020-07-30', '2020-08-1')
    .sort('CLOUD_COVER')
    .first()
    .select('SR_B[2-7]')
)

print(ee.Date(landsat_prefire_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()) 
landsat_prefire_image = landsat_prefire_image.select('SR_B.').multiply(0.0000275).add(-0.2)

vis_params = {'min':0, 'max':0.4, 'bands': ['SR_B7', 'SR_B5', 'SR_B4']} # swir2, nir, red

landsatMap.centerObject(point, 9)
landsatMap.addLayer(landsat_prefire_image, vis_params, "Prefire")
# landsatMap

2020-07-31


### Calculating Prefire NBR

In [26]:
prefire_nbr = landsat_prefire_image.expression(
    '(NIR - SWIR) / (NIR + SWIR)', {
        'NIR': landsat_prefire_image.select('SR_B5'),
        'SWIR': landsat_prefire_image.select('SR_B7'),
    })

# Add the image layer to the map and display it.
landsatMap.add_ee_layer(prefire_nbr, {
    # 'min': -1,
    # 'max': 1,
    'palette': ['#d7191c', '#fdae61','#ffffbf', '#a6d96a','#1a9641']
}, 'prefire_nbr_map')

# landsatMap

## Postfire

In [27]:
point = ee.Geometry.Point([-122.8, 39.87])

# obtaining postfire image of landsat
landsat_postfire_image = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(point)
    .filterDate('2020-10-18', '2020-10-21')
    .sort('CLOUD_COVER')
    .first()
    .select('SR_B[2-7]')
)
# printing the bands present in the iamge
print(landsat_postfire_image.bandNames().getInfo())
# printing date of image
print(ee.Date(landsat_postfire_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()) 
# converting DN into radiance
landsat_postfire_image = landsat_postfire_image.select('SR_B.').multiply(0.0000275).add(-0.2)

vis_params = {'min':0, 'max':0.4, 'bands': ['SR_B7', 'SR_B5', 'SR_B4']} # swir2, nir, red

landsatMap.centerObject(point, 9)
landsatMap.addLayer(landsat_postfire_image, vis_params, "Postfire")

# display the map
# landsatMap.addLayerControl()
# landsatMap

['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
2020-10-19


### calculating Postfire NBR

In [28]:
postfire_nbr = landsat_postfire_image.expression(
    '(NIR - SWIR) / (NIR + SWIR)', {
        'NIR': landsat_postfire_image.select('SR_B5'),
        'SWIR': landsat_postfire_image.select('SR_B7'),
    })

# Define a map centered the california
# postfire_nbr_map = folium.Map(location=[40,-123], zoom_start=10)
landsatMap.centerObject(point, 9)

# Add the image layer to the map and display it.
landsatMap.add_ee_layer(postfire_nbr, {
    # 'min': -1,
    # 'max': 1,
    'palette': ['#d7191c', '#fdae61','#ffffbf', '#a6d96a','#1a9641']
}, 'postfire_nbr_map')

landsatMap.addLayerControl()
landsatMap

Map(center=[39.87, -122.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

## d_NBR

In [29]:
d_nbr = prefire_nbr.subtract(postfire_nbr)

# Define a map 
landsatMap.add_ee_layer(d_nbr, 
    {
#     'min': -0.5,
#     'max': 1.3,
#     'palette': ['#1a9641', '#a6d96a', '#ffffbf','#fdae61','#d7191c']
                        },
    'd_nbr_map')

landsatMap.addLayerControl()
landsatMap

Map(center=[39.87, -122.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [20]:
# legend_dict = {
#     'very high': 'd7191c',
#     'high': 'fdae61',
#     'medium': 'ffffbf',
#     'low': 'a6d96a',
#     'very low': '1a9641',
# }

# landsatMap.add_legend(legend_title="NBR", legend_dict=legend_dict)

## Adding dNBR map to the bands of Landsat-8 Postfire

In [31]:
d_nbr = d_nbr.rename('dNBR')
landsat_postfire_image = landsat_postfire_image.addBands(d_nbr)
print(landsat_postfire_image.bandNames().getInfo())

['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'dNBR']


In [32]:
# legend_dict = {
#     'very low': 'd7191c',
#     'low': 'fdae61',
#     'medium': 'ffffbf',
#     'high': 'a6d96a',
#     'very high': '1a9641',
# }

landsatMap.add_legend(legend_title="dNBR", legend_dict=legend_dict)
landsatMap

Map(bottom=25003.090911865234, center=[40.451127265872316, -122.57995605468751], controls=(WidgetControl(optio…

# WARNING: Draw Geometries for Fire Region here, then Proceed!

---

In [33]:
landsatMap.draw_features

In [34]:
fire_lst_geom = landsatMap.draw_features

fireROI = fire_lst_geom[0]

for i in range(1, len(fire_lst_geom)):  
    fireROI = fireROI.union(fire_lst_geom[i], maxError=1)
print('No. of polygons made roughly equal to ', i)

No. of polygons made roughly equal to  29


In [35]:
landsatMap.add_ee_layer(fireROI, {}, 'fire_ROI')

# WARNING: Draw Geometries for Non-Fire Region here, then Proceed!

---

In [36]:
nonFire_lst_geom = landsatMap.draw_features

nonFireROI = nonFire_lst_geom[0]
for i in range(1, len(nonFire_lst_geom)):  
    nonFireROI = nonFireROI.union(nonFire_lst_geom[i], maxError=1)
print('No. of polygons made roughly equal to ', i)

No. of polygons made roughly equal to  10


In [37]:
landsatMap.add_ee_layer(nonFireROI, {}, 'forest_ROI')

In [38]:
# set properties to the features
fireROI = fireROI.set('class', 0)
nonFireROI = nonFireROI.set('class', 1)

# Make a feature collection from the hand-made geometries
polygons = ee.FeatureCollection([
  fireROI,
  nonFireROI
])

polygons = polygons.map(ee.Feature)

nonFire1_img = landsat_postfire_image.clip(nonFireROI)
fire1_img = landsat_postfire_image.clip(fireROI)

vis = {'bands': ['SR_B7', 'SR_B5', 'SR_B4'], 'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]
    }

In [39]:
# polygons['columns'].getInfo
print(polygons.getInfo())

{'type': 'FeatureCollection', 'columns': {'class': 'Integer', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-123.273728, 40.09519300000001], [-123.271855, 40.09519300000001], [-123.271855, 40.096417], [-123.273728, 40.09641700000001], [-123.273728, 40.09519300000001]]], [[[-123.30244399999998, 40.172127], [-123.298464, 40.172127], [-123.298464, 40.17469], [-123.30244399999998, 40.17469], [-123.30244399999998, 40.172127]]], [[[-123.255974, 40.097879], [-123.253282, 40.097879], [-123.253282, 40.100504], [-123.255974, 40.100504], [-123.255974, 40.097879]]], [[[-123.299791, 40.179577], [-123.295421, 40.179577], [-123.295421, 40.182736], [-123.299791, 40.182736], [-123.299791, 40.179577]]], [[[-123.237451, 40.081281], [-123.23120799999998, 40.081281], [-123.23120799999998, 40.08522], [-123.237451, 40.08522], [-123.237451, 40.081281]]], [[[-123.285041, 40.18845699999999], [-123.27848600000002, 40.18845699999999], [-123.278

In [40]:
landsatMap.add_ee_layer(nonFire1_img, vis, 'Non Fire')
landsatMap.add_ee_layer(fire1_img, vis, 'Fire')

landsatMap

Map(bottom=49771.0, center=[40.31579194870286, -122.75896645779824], controls=(WidgetControl(options=['positio…

## Classification Process

In [41]:
related_bands = ['SR_B4','SR_B5','SR_B6','SR_B7','dNBR']

In [42]:
#Get the values for all the pixels in each polygon in the training
 
trainImage = landsat_postfire_image.sampleRegions(**{
    # Get the sample from the polygons FeatureCollection
    'collection': polygons,
    # Keep this list of properties from the polygons
     'properties': ['class'],
      #Set the scale to get Landsat pixels in the polygons
      'scale': 30
})

In [43]:
trainingData = trainImage.randomColumn()
trainSet = trainingData.filter('random<0.8')
testSet = trainingData.filter('random>=0.8')

In [44]:
# print('No. of pixels in training set:', len(trainSet.getInfo()['features']))
# print('No. of pixels in test set:', len(testSet.getInfo()['features']))

In [45]:
# Create an SVM classifier with custom parameters
model = ee.Classifier.libsvm(**{
    'decisionProcedure':'Margin',
    'kernelType': 'RBF',
    'gamma': 0.5,
    'cost': 1
})

In [46]:
# Train the classifier
svm_classifier = model.train(trainSet, 'class', related_bands)

In [47]:
print('Results of trained classifier', svm_classifier.explain().getInfo())

Results of trained classifier {'classes': [0, 1], 'svm_parameters': {'C': 1, 'coef0': 0, 'degree': 0, 'eps': 0.001, 'gamma': 0.5, 'kernel_type': 'RBF', 'nu': 0.5, 'p': 0.1, 'probability': 0, 'shrinking': 1, 'svm_type': 'C_SVC'}}


In [48]:
# Classify the image
classified = landsat_postfire_image.classify(**{
                        'classifier': svm_classifier,
                        'outputName':'predicted_landcover'}) 

In [49]:
landsatMap.add_ee_layer(classified, {'min': 0, 'max': 1, 'palette': ['red', 'green']}, 'deforestation')
landsatMap

Map(bottom=49771.0, center=[40.31579194870286, -122.75896645779824], controls=(WidgetControl(options=['positio…

In [50]:
# Create a folium map object.
my_map = folium.Map(location=[39.87,-123.090], zoom_start = 8)

vis_params = {'min':0, 'max':0.4, 'bands': ['SR_B7', 'SR_B5', 'SR_B4']} # swir2, nir, red
my_map.add_ee_layer(landsat_postfire_image, vis_params, "Postfire")
my_map.add_ee_layer(classified, {'min': 0, 'max': 1, 'palette': ['red', 'green']}, 'deforestation')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

### Training Accuracy Assessment with Confusion Matrix

In [51]:
# Get a confusion matrix and overall accuracy for the training sample.
confMatrixTrain = svm_classifier.confusionMatrix()
# print('Training error matrix', trainAccuracy)

In [52]:
# print(type(confMatrixTrain))
print(confMatrixTrain.getInfo())

[[3379, 23], [179, 45437]]


In [53]:
# // Calculate overall accuracy.
print("Overall accuracy =", confMatrixTrain.accuracy().getInfo())

# // Calculate consumer's accuracy, also known as user's accuracy or
# // specificity and the complement of commission error (1 − commission error).
print("Consumer's accuracy =", confMatrixTrain.consumersAccuracy().getInfo())

# // Calculate producer's accuracy, also known as sensitivity and the
# // compliment of omission error (1 − omission error).
print("Producer's accuracy =", confMatrixTrain.producersAccuracy().getInfo())

# // Calculate kappa statistic.
print('Kappa statistic =', confMatrixTrain.kappa().getInfo())

Overall accuracy = 0.9958790648333266
Consumer's accuracy = [[0.9496908375491849, 0.9994940607127145]]
Producer's accuracy = [[0.9932392710170488], [0.9960759382672746]]
Kappa statistic = 0.9687602829520081


### Testing 

In [54]:
testRes = testSet.classify(**{
                        'classifier': svm_classifier,
                        'outputName': 'predicted_landcover'
})

# em is error_matrix 
em = testRes.errorMatrix(**{
                        'actual':'class',
                        'predicted': 'predicted_landcover'
                    })

confMatrixTest = ee.ConfusionMatrix(em)

In [55]:
print(confMatrixTest.getInfo())

[[895, 7], [37, 11559]]


In [56]:
# // Calculate overall accuracy.
print("Overall accuracy =", confMatrixTest.accuracy().getInfo())

# // Calculate consumer's accuracy, also known as user's accuracy or
# // specificity and the complement of commission error (1 − commission error).
print("Consumer's accuracy =", confMatrixTest.consumersAccuracy().getInfo())

# // Calculate producer's accuracy, also known as sensitivity and the
# // compliment of omission error (1 − omission error).
print("Producer's accuracy =", confMatrixTest.producersAccuracy().getInfo())

# // Calculate kappa statistic.
print('Kappa statistic =', confMatrixTest.kappa().getInfo())

Overall accuracy = 0.9964794367098736
Consumer's accuracy = [[0.9603004291845494, 0.9993947777969912]]
Producer's accuracy = [[0.9922394678492239], [0.9968092445670921]]
Kappa statistic = 0.9741096093955539
